In [5]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import metrics
from read_tfrecord import *
from mtcnn_model import Onet,cls_ohem,cal_accuracy,bbox_ohem
from tqdm import tqdm
import cv2

In [6]:
data_path = "data/48/train_ONet_landmark.tfrecord_shuffle"

In [9]:
# 加载pokemon数据集的工具！
def load_pokemon(mode='train'):
    """ 加载pokemon数据集的工具！
    :param root:    数据集存储的目录
    :param mode:    mode:当前加载的数据是train,val,还是test
    :return:
    """
    # # 创建数字编码表,范围0-4;
    # name2label = {}  # "sq...":0   类别名:类标签;  字典 可以看一下目录,一共有5个文件夹,5个类别：0-4范围;
    # for name in sorted(os.listdir(os.path.join(root))):     # 列出所有目录;
    #     if not os.path.isdir(os.path.join(root, name)):
    #         continue
    #     # 给每个类别编码一个数字
    #     name2label[name] = len(name2label.keys())
 
    # 读取Label信息;保存索引文件images.csv
    # [file1,file2,], 对应的标签[3,1] 2个一一对应的list对象。
    # 根据目录,把每个照片的路径提取出来,以及每个照片路径所对应的类别都存储起来，存储到CSV文件中。
    size = 48
    images,labels,boxes = red_tf(data_path,size)
 
    # 图片切割成，训练70%，验证15%，测试15%。
    if mode == 'train':                                                     # 100% 训练集
        images = images[:int(len(images))]
        labels = labels[:int(len(labels))]
        boxes  = boxes[:int(len(boxes))]
    elif mode == 'val':                                                     # 15% = 70%->85%  验证集
        images = images[int(0.7 * len(images)):int(0.85 * len(images))]
        labels = labels[int(0.7 * len(labels)):int(0.85 * len(labels))]
        boxes = boxes[int(0.7 * len(boxes)):int(0.85 * len(boxes))]
    else:                                                                   # 15% = 70%->85%  测试集
        images = images[int(0.85 * len(images)):]
        labels = labels[int(0.85 * len(labels)):]
        boxes = boxes[int(0.85 * len(boxes)):]
    ima = tf.data.Dataset.from_tensor_slices(images)
    lab = tf.data.Dataset.from_tensor_slices(labels)
    roi = tf.data.Dataset.from_tensor_slices(boxes)
 
    train_data = tf.data.Dataset.zip((ima, lab, roi)).shuffle(1000).batch(32)
    train_data = list(train_data.as_numpy_iterator())
    return train_data
 
# 图像色相变换
def image_color_distort(inputs):
    inputs = tf.image.random_contrast(inputs, lower=0.5, upper=1.5)
    inputs = tf.image.random_brightness(inputs, max_delta=0.2)
    inputs = tf.image.random_hue(inputs,max_delta= 0.2)
    inputs = tf.image.random_saturation(inputs,lower = 0.5, upper= 1.5)
    return inputs
 
 
 
def train(eopch):
    model = Onet()
    #model.load_weights("onet.h5")
 
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    off = 1000
    acc_meter = metrics.Accuracy()
    for epoch in tqdm(range(eopch)):
 
        for i,(img,lab,boxes) in enumerate(load_pokemon("train")):
 
 
            img = image_color_distort(img)
            # 开一个gradient tape, 计算梯度
            with tf.GradientTape() as tape:
                #cls_prob, bbox_pred,laim = model(img)
                cls_prob, bbox_pred = model(img)
                
                cls_loss = cls_ohem(cls_prob, lab)
                bbox_loss = bbox_ohem(bbox_pred, boxes,lab)
                # landmark_loss = landmark_loss_fn(landmark_pred, landmark_batch, label_batch)
                # accuracy = cal_accuracy(cls_prob, label_batch)
                
                '''
                print("lab",lab)
                print("cls_prob",cls_prob)
                print("bbox_pred",bbox_pred)
                print("cls_losss",cls_loss)
                print("bbox_loss",bbox_loss)
                '''
    
                total_loss_value = cls_loss + 0.5 * bbox_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('Training loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 6))
           
 
        for i, (v_img, v_lab1, boxes) in enumerate(load_pokemon("val")):
            v_img = image_color_distort(v_img)
            with tf.GradientTape() as tape:
                #cls_prob, bbox_pred,laim= model(v_img)
                cls_prob, bbox_pred = model(v_img)
                cls_loss = cls_ohem(cls_prob, v_lab1)
                bbox_loss = bbox_ohem(bbox_pred, boxes,v_lab1)
                # landmark_loss = landmark_loss_fn(landmark_pred, landmark_batch, label_batch)
                # accuracy = cal_accuracy(cls_prob, label_batch)
 
 
                total_loss_value = cls_loss + 0.5 * bbox_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('val___ loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 6))
    model.save_weights('./Weights/Onet_wight/onet_30.ckpt')

In [10]:
train(30)

  0%|          | 0/30 [00:00<?, ?it/s]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
square_error tf.Tensor([0.07678535 0.07097137 0.04287487], shape=(3,), dtype=float32)
Training loss (for one batch) at step 0: 0.7118601202964783
Seen so far: 6 samples
square_error tf.Tensor([0.23676684 0.17796806 0.12847695 0.08177916 0.03292888 0.01822644], shape=(6,), dtype=float32)
square_error tf.Tensor([0.24147303 0.09191622 0.03954558 0.02827022], shape=(4,), dtype=float32)
square_error tf.Tensor(
[0.3909358  0.2913944  0.24822864 0.15246716 0.1188772  0.09115031
 0.08895896 0.04581497 0.04292878 0.02456921], shape=(10,), dtype=float32)
square_error tf.Tensor(
[0.4462828  0.38474315 0.36020628 0.09934627 0.08985039 0.04859312
 0.04407709 0.00566783], shape=(8,), dtype=float32)
square_error tf.Tensor(
[0.28836247 0.14962988 0.065613   0.04653832 0.02474059 0.01854351
 0.00936185 0.00519966 0.00380862], shape=(9,), dtype=float

square_error tf.Tensor([0.09200732 0.0621612  0.04296335 0.02843357 0.02790595], shape=(5,), dtype=float32)
square_error tf.Tensor(
[0.31027105 0.17158844 0.14352134 0.10796618 0.08466439 0.04469061
 0.03875722 0.03868291 0.00920531], shape=(9,), dtype=float32)
square_error tf.Tensor([0.21960708 0.16449177 0.0259252  0.00742037], shape=(4,), dtype=float32)
square_error tf.Tensor(
[0.26835036 0.1181673  0.10653965 0.0631796  0.02802649 0.02095029
 0.02032779 0.01779719], shape=(8,), dtype=float32)
square_error tf.Tensor([0.11997604 0.08912142 0.06808368 0.06562459 0.03261999 0.0102314 ], shape=(6,), dtype=float32)
square_error tf.Tensor([0.12074578 0.12046927 0.07269362 0.04914233 0.03494003], shape=(5,), dtype=float32)
square_error tf.Tensor([0.17795895 0.14853811 0.12420852 0.06252405 0.00667755], shape=(5,), dtype=float32)
square_error tf.Tensor(
[0.15581387 0.1363144  0.09611634 0.07330018 0.07195237 0.06213045
 0.06103948], shape=(7,), dtype=float32)
square_error tf.Tensor([0.08530

square_error tf.Tensor(
[0.19071461 0.13777146 0.09280843 0.06424432 0.04026788 0.01682247
 0.00538027], shape=(7,), dtype=float32)
square_error tf.Tensor([0.26274973 0.05706327 0.0112209 ], shape=(3,), dtype=float32)
square_error tf.Tensor([0.15653926 0.08534528 0.06761396 0.06410215 0.03498755 0.01616204], shape=(6,), dtype=float32)
square_error tf.Tensor(
[0.35350102 0.29866883 0.1581121  0.07284621 0.0500531  0.03604512
 0.03144712 0.02708241 0.0186276 ], shape=(9,), dtype=float32)
square_error tf.Tensor(
[0.26064485 0.12190174 0.08613892 0.08038958 0.07305802 0.06610367
 0.06107293 0.03226496 0.02762768 0.00999152], shape=(10,), dtype=float32)
square_error tf.Tensor([0.2991949  0.13773231 0.11900304 0.04872686 0.0159789  0.0151592 ], shape=(6,), dtype=float32)
square_error tf.Tensor([0.29057235 0.16055734 0.06941298 0.06440744 0.02524849 0.01746724], shape=(6,), dtype=float32)
square_error tf.Tensor([0.18296379 0.12195332 0.06631844 0.0590184  0.04546763 0.0173769 ], shape=(6,), d

square_error tf.Tensor(
[0.39615962 0.36359042 0.16632919 0.14560914 0.06827086 0.06539105
 0.0593577  0.05568001 0.05237913 0.01053587], shape=(10,), dtype=float32)
square_error tf.Tensor([0.04785949 0.03723935 0.00882254], shape=(3,), dtype=float32)
square_error tf.Tensor([0.18651664 0.17932615 0.0710206  0.0534924  0.0437987  0.02680638], shape=(6,), dtype=float32)
square_error tf.Tensor(
[0.14813174 0.11124931 0.07605445 0.04967587 0.04712288 0.02580982
 0.02300578 0.01393392 0.01305116], shape=(9,), dtype=float32)
square_error tf.Tensor(
[0.2429589  0.18355004 0.09075618 0.06913725 0.01388793 0.01311396
 0.00825576], shape=(7,), dtype=float32)
square_error tf.Tensor([0.12696375 0.09077314 0.08189211 0.07871793 0.01215833], shape=(5,), dtype=float32)
square_error tf.Tensor(
[0.1957506  0.164389   0.10695488 0.09744485 0.048534   0.04028168
 0.03514348 0.02700691 0.01031509 0.00286067 0.00212355], shape=(11,), dtype=float32)
square_error tf.Tensor([0.11516905 0.08484238 0.05555888 0

...

square_error tf.Tensor([0.05300634 0.04606356 0.04497441 0.02841697 0.02176471 0.02013205], shape=(6,), dtype=float32)
square_error tf.Tensor([0.04068339 0.02673686 0.0079468  0.0045781 ], shape=(4,), dtype=float32)
square_error tf.Tensor([0.05243767 0.05101377 0.02594151 0.01964956], shape=(4,), dtype=float32)
square_error tf.Tensor([0.10752378 0.052271   0.04752051 0.013714   0.01016835], shape=(5,), dtype=float32)
square_error tf.Tensor(
[0.30298406 0.08626541 0.01992955 0.01938107 0.01450924 0.01405553
 0.01324121], shape=(7,), dtype=float32)
square_error tf.Tensor(
[0.0481352  0.03891169 0.0319797  0.02441312 0.01905318 0.01172916
 0.00807035], shape=(7,), dtype=float32)
square_error tf.Tensor([0.04491892 0.02881761 0.01878772 0.0120398 ], shape=(4,), dtype=float32)
square_error tf.Tensor([0.39394695 0.14299318 0.08149143 0.01896646 0.00766093], shape=(5,), dtype=float32)
square_error tf.Tensor([0.08154795 0.02079153 0.01937227 0.01357981], shape=(4,), dtype=float32)
square_error 

square_error tf.Tensor(
[0.12961832 0.12758811 0.06413599 0.03854631 0.00556342 0.00436278
 0.00410875], shape=(7,), dtype=float32)
square_error tf.Tensor(
[0.11941881 0.02951204 0.0178321  0.01336239 0.01154833 0.00851658
 0.00355156], shape=(7,), dtype=float32)
square_error tf.Tensor([0.05388501 0.04237262 0.03686329 0.02474821 0.00958618 0.00596159], shape=(6,), dtype=float32)
square_error tf.Tensor([0.0728342  0.0380136  0.02392361 0.02197111 0.0124792  0.00453663], shape=(6,), dtype=float32)
square_error tf.Tensor([0.11639789 0.05391091 0.04844125 0.01934242 0.01901679 0.0174475 ], shape=(6,), dtype=float32)
square_error tf.Tensor(
[0.23082227 0.11774088 0.07010184 0.04314906 0.01721465 0.0137112
 0.00988054 0.00728517], shape=(8,), dtype=float32)
square_error tf.Tensor([0.14317973 0.03669161 0.03437084 0.03308656 0.00682751 0.00436782], shape=(6,), dtype=float32)
square_error tf.Tensor(
[0.13424988 0.08758597 0.05955701 0.05145343 0.04716393 0.03249216
 0.00415976], shape=(7,), d

square_error tf.Tensor(
[0.10451014 0.08516448 0.06382638 0.06110187 0.03709537 0.02848412
 0.01934174 0.01170429 0.01075536], shape=(9,), dtype=float32)
square_error tf.Tensor(
[0.04409046 0.04244504 0.03103103 0.02818418 0.0259151  0.02041457
 0.01992792 0.01880375 0.0098662  0.00073458], shape=(10,), dtype=float32)
square_error tf.Tensor([0.07769081 0.03512951 0.03173649 0.03000049 0.00697527 0.00225159], shape=(6,), dtype=float32)
square_error tf.Tensor(
[0.08371855 0.05932888 0.04849828 0.04103894 0.04062512 0.00772267
 0.00685955 0.00670015 0.00131241], shape=(9,), dtype=float32)
square_error tf.Tensor(
[0.10108183 0.05629454 0.05010708 0.04000352 0.01778249 0.01417129
 0.01364577 0.01011687 0.00835735 0.00327213], shape=(10,), dtype=float32)
square_error tf.Tensor([0.03732915 0.00686028], shape=(2,), dtype=float32)
square_error tf.Tensor([0.06158154 0.03575655 0.0232925  0.02259995 0.01511171 0.0131805 ], shape=(6,), dtype=float32)
square_error tf.Tensor(
[0.07887501 0.05690941 

100%|██████████| 30/30 [26:19<00:00, 52.66s/it]

square_error tf.Tensor(
[0.06993162 0.05941938 0.05542403 0.04898393 0.04015085 0.02676286
 0.01648402 0.01611712 0.01301398 0.00484956], shape=(10,), dtype=float32)
square_error tf.Tensor(
[0.06301781 0.05154343 0.04227803 0.04107328 0.03977221 0.03159108
 0.01939361 0.01296303], shape=(8,), dtype=float32)
